# BlazingSQL vs. Apache Spark 

Below we have one of our popular workloads running with [BlazingSQL](https://blazingsql.com/) + [RAPIDS AI](https://rapids.ai) and then running the entire ETL phase again, only this time with Apache Spark + PySpark.

In this notebook, we will cover: 
- How to set up BlazingSQL and the RAPIDS AI suite in Google Colab.
- How to read and query csv files with cuDF and BlazingSQL.
- How BlazingSQL compares against Apache Spark (analyzing over 20M records).

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-39814657-5&cid=555&t=event&ec=guides&ea=bsql_vs_spark&dt=bsql_vs_spark)

## Setup
### Environment Sanity Check 

RAPIDS packages (BlazingSQL included) require Pascal+ architecture to run. For Colab, this translates to a T4 GPU instance. 

The cell below will let you know what type of GPU you've been allocated, and how to proceed.

In [14]:
!wget https://github.com/BlazingDB/bsql-demos/raw/master/utils/colab_env.py
!python colab_env.py 



***********************************
GPU = b'Tesla T4'
Woo! You got the right kind of GPU!
***********************************




## Installs 
The cell below pulls our Google Colab install script from the `bsql-demos` repo then runs it. The script first installs miniconda, then uses miniconda to install BlazingSQL and RAPIDS AI. This takes a few minutes to run. 

In [ ]:
!wget https://github.com/BlazingDB/bsql-demos/raw/master/utils/bsql-colab.sh 
!bash bsql-colab.sh

import sys, os, time
sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

import subprocess
subprocess.Popen(['blazingsql-orchestrator', '9100', '8889', '127.0.0.1', '8890'],stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
subprocess.Popen(['java', '-jar', '/usr/local/lib/blazingsql-algebra.jar', '-p', '8890'])

import pyblazing.apiv2.context as cont
cont.runRal()
time.sleep(1) 

## Import packages and create Blazing Context
You can think of the BlazingContext much like a Spark Context (i.e. where information such as FileSystems you have registered and Tables you have created will be stored). If you have issues running this cell, restart runtime and try running it again.

In [1]:
from blazingsql import BlazingContext
import cudf
# start up BlazingSQL
bc = BlazingContext()

BlazingContext ready


### Load & Query Table
First, we need to download the netflow data (20 million records) from AWS.

In [ ]:
# takes a few minutes to download
!wget https://blazingsql-colab.s3.amazonaws.com/netflow_data/nf-chunk2.csv

#### BlazingSQL + cuDF 
Data in hand, we can test the preformance of cuDF and BlazingSQL on this dataset.

In [2]:
%%time
# Load CSVs into GPU DataFrames (GDF)
netflow_gdf = cudf.read_csv('nf-chunk2.csv')

CPU times: user 138 ms, sys: 142 ms, total: 280 ms
Wall time: 304 ms


In [3]:
%%time
# Create BlazingSQL table from GDF - There is no copy in this process
bc.create_table('netflow', netflow_gdf)

CPU times: user 27.5 ms, sys: 747 µs, total: 28.2 ms
Wall time: 55.9 ms


In [4]:
%%time
# make a query
query = '''
        SELECT
            a.firstSeenSrcIp as source,
            a.firstSeenDestIp as destination,
            count(a.firstSeenDestPort) as targetPorts,
            SUM(a.firstSeenSrcTotalBytes) as bytesOut,
            SUM(a.firstSeenDestTotalBytes) as bytesIn,
            SUM(a.durationSeconds) as durationSeconds,
            MIN(parsedDate) as firstFlowDate,
            MAX(parsedDate) as lastFlowDate,
            COUNT(*) as attemptCount
        FROM
            netflow a
        GROUP BY
            a.firstSeenSrcIp,
            a.firstSeenDestIp
            '''

# query the table
gdf = bc.sql(query)

CPU times: user 30.8 ms, sys: 0 ns, total: 30.8 ms
Wall time: 429 ms


In [5]:
# how's it look?
gdf.head()

,source,destination,targetPorts,bytesOut,bytesIn,durationSeconds,firstFlowDate,lastFlowDate,attemptCount
0,172.30.2.48,172.0.0.1,1,90,0,0,2013-04-03 06:36:09,2013-04-03 06:36:09,1
1,172.10.2.81,239.255.255.250,14,2800,0,6,2013-04-03 06:36:41,2013-04-03 06:36:48,14
2,172.30.2.58,172.0.0.1,1,90,0,0,2013-04-03 06:36:09,2013-04-03 06:36:09,1
3,172.30.1.171,10.0.0.13,1,454,633,0,2013-04-03 06:48:02,2013-04-03 06:48:02,1
4,172.30.1.17,10.0.0.7,1,453,632,0,2013-04-03 06:47:56,2013-04-03 06:47:56,1


## Apache Spark
The cell below installs Apache Spark ([PySpark](https://spark.apache.org/docs/latest/api/python/index.html)).

In [4]:
# Note: This installs Spark (version 2.4.1, as tested in Jan 2020)
!pip install pyspark

#### PyBlazing vs PySpark
With everything installed we can launch a SparkSession and see how BlazingSQL stacks up.

In [6]:
%%time
# I copied this cell's snippet from another Google Colab by Luca Canali here: https://colab.research.google.com/github/LucaCanali/sparkMeasure/blob/master/examples/SparkMeasure_Jupyter_Colab_Example.ipynb

from pyspark.sql import SparkSession

# Create Spark Session
# This example uses a local cluster, you can modify master to use  YARN or K8S if available 
# This example downloads sparkMeasure 0.13 for scala 2_11 from maven central

spark = SparkSession \
 .builder \
 .master("local[*]") \
 .appName("PySpark Netflow Benchmark code") \
 .config("spark.jars.packages","ch.cern.sparkmeasure:spark-measure_2.11:0.13")  \
 .getOrCreate()

CPU times: user 50.2 ms, sys: 12.9 ms, total: 63.1 ms
Wall time: 3.88 s


### Load & Query Table

In [5]:
%%time
# load CSV into Spark
netflow_df = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('nf-chunk2.csv')

CPU times: user 2.73 ms, sys: 0 ns, total: 2.73 ms
Wall time: 2.91 s


In [6]:
%%time
# create table for querying
netflow_df.createOrReplaceTempView('netflow')

CPU times: user 1.06 ms, sys: 611 µs, total: 1.67 ms
Wall time: 120 ms


In [7]:
%%time
# make a query
query = '''
        SELECT
            a.firstSeenSrcIp as source,
            a.firstSeenDestIp as destination,
            count(a.firstSeenDestPort) as targetPorts,
            SUM(a.firstSeenSrcTotalBytes) as bytesOut,
            SUM(a.firstSeenDestTotalBytes) as bytesIn,
            SUM(a.durationSeconds) as durationSeconds,
            MIN(parsedDate) as firstFlowDate,
            MAX(parsedDate) as lastFlowDate,
            COUNT(*) as attemptCount
        FROM
            netflow a
        GROUP BY
            a.firstSeenSrcIp,
            a.firstSeenDestIp
            '''

# query with Spark
edges_df = spark.sql(query)

# set/display results
edges_df.show(5)

+------------+---------------+-----------+--------+-------+---------------+-------------------+-------------------+------------+
|      source|    destination|targetPorts|bytesOut|bytesIn|durationSeconds|      firstFlowDate|       lastFlowDate|attemptCount|
+------------+---------------+-----------+--------+-------+---------------+-------------------+-------------------+------------+
| 172.10.1.13|239.255.255.250|         15|    2975|      0|              6|2013-04-03 06:36:19|2013-04-03 06:36:27|          15|
|172.10.1.232|      172.0.0.1|          1|     180|    180|              0|2013-04-03 06:36:45|2013-04-03 06:36:45|           1|
|172.10.1.238|239.255.255.250|          2|     700|      0|              6|2013-04-03 06:36:44|2013-04-03 06:36:51|           2|
| 172.10.1.35|      172.0.0.1|          1|     270|      0|              0|2013-04-03 06:36:21|2013-04-03 06:36:21|           1|
|172.10.2.137|      172.0.0.1|          1|      90|     90|              0|2013-04-03 06:36:42|20